# OpenAI Whisper API, Sentiment Analysis, Summarization

### Installing Required Libraries

In [1]:
!pip install -q openai
!pip install vaderSentiment
!pip install -q langchain
!pip install -q docx2txt

### Load Environment Variables

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

print("API Key Loaded", os.environ.get('OPENAI_API_KEY') is not None)

API Key Loaded True


### Import Libraries

In [5]:
from openai import OpenAI
from openai import Audio

### Creating OpenAI Client

In [7]:
client = OpenAI()

### Transcribe Function

In [8]:
def transcribe(audio_file, not_english=False):
    if not os.path.exists(audio_file):
        print('Audio file provided does not exist!')
        return False
    if not_english:
        print('Starting translating into English ...', end='')
        transcription=client.audio.translations.create(
            model='whisper-1',
            file=open(audio_file, 'rb')
        )
        print('Translation and transcription completed.')
    else:
        print('Starting transcription ...', end='')
        transcription = client.audio.transcriptions.create(
            model='whisper-1',
            file = open(audio_file, 'rb')
        )
        print('Transcription Completed!')

    name, extension = os.path.splitext(audio_file)
    transcript_filename = f'transcript-{name}.txt'
    with open(transcript_filename, 'w') as f:
        f.write(transcription.text)

    return transcript_filename

### Testing the application

In [15]:
transcript_filename = transcribe('Sample03.m4a')
print(transcript_filename)

Starting transcription ...Transcription Completed!
transcript-Sample03.txt


In [16]:
with open(transcript_filename) as f:
    print(f.read())

Okay, let's get this project planning meeting started. Cindy, can you recap the latest client feedback? Mark, where do we stand on the design deliverable? Sarah, were there any updates on the budget projection? We need to solidify our timeline for the next two weeks to stay on track. Any potential roadblocks we should discuss? Cindy, sure, the client really liked the initial design concept we presented, but they requested for a few minor tweaks. They would like to see the logo a bit bigger and explore more alternative color schemes. Mark, your turn. The design team is finalized with the deliverable now. We should have everything ready for the review by the end of the day. Now your turn, Sarah. Ah, no major changes on the budget front. Everything seemed to be on track according to the latest projection. Alright, great. Let's put together a revised timeline incorporating these updates and circulate it to everyone by lunchtime. Are there any potential roadblocks that might hold us up? Mar

In [19]:
transcript_filename = transcribe('ArabicSample01.m4a', not_english=True)
print(transcript_filename)
with open(transcript_filename) as f:
    print(f.read())

Starting translating into English ...Translation and transcription completed.
transcript-ArabicSample01.txt
This is a note to remind you to upgrade every .NET project from version 5 to version 8 and to use the Blazor server client The upgrade will not be easy, it will take a lot of time and I have to upgrade the packages one by one to make sure that the project is working properly and I have to work on the development environment and staging If everything is fine, I will start the production


### Batch Processing Voice Memos

In [20]:
def transcribe_multiple(audio_files):
    transcripts = []
    for audio_file in audio_files:
        transcript_file= transcribe(audio_file)
        transcripts.append(transcript_file)
    return transcripts

In [22]:
transcripts_combined = transcribe_multiple(['ArabicSample01.m4a','Sample03.m4a'])

Starting transcription ...Transcription Completed!
Starting transcription ...Transcription Completed!


In [23]:
print(transcripts_combined)

['transcript-ArabicSample01.txt', 'transcript-Sample03.txt']


In [24]:
for transcript in transcripts_combined:
    with open(transcript) as f:
        print(f'Memo :{transcript}')
        print(f.read())

Memo :transcript-ArabicSample01.txt
هذا الملاحظة لكي أتذكر أنني أعمل تغيير لكل مشروع توتنات من فرجن 5 إلى فرجن 8 ولكي أستخدم بلايزر بلايزر سيرفر وكليانت التغيير لن يكون سهل سيأخذ الكثير من الوقت ويجب أن أعمل تغيير لكل المشروعات لكي أتأكد أن المشروع يعمل بشكل محق ويجب أن أعمل تغيير هذا على البيئة التنفيذية ثم التغيير إذا كان كل شيء بخير سأقوم بعمل التغيير
Memo :transcript-Sample03.txt
Okay, let's get this project planning meeting started. Cindy, can you recap the latest client feedback? Mark, where do we stand on the design deliverable? Sarah, were there any updates on the budget projection? We need to solidify our timeline for the next two weeks to stay on track. Any potential roadblocks we should discuss? Cindy, sure, the client really liked the initial design concept we presented, but they requested for a few minor tweaks. They would like to see the logo a bit bigger and explore more alternative color schemes. Mark, your turn. The design team is finalized with the deliverable now. We

### Sentiment Intensity Analyzer

In [29]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
def analyze_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(text)

    if sentiment_scores['compound'] >= 0.05:
        return 'Positive'
    elif sentiment_scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'

In [30]:
sentiment_result = analyze_sentiment(transcript_filename)
print(sentiment_result)

Neutral


### Summarization Using GPT

In [34]:
def summarize(transcript_filename):
    if not os.path.exists(transcript_filename):
        print('The transcript file does not exist!')
        return False
    with open(transcript_filename) as f:
        transcript = f.read()

    system_prompt = 'I want you to act as my creative assistant'
    prompt=f'''
    Create a summary of the following text.
    Text:{transcript}

    1. Add a title to the summary
    2. Your summary should be informative and cover the most important aspect of the topic.
    3. Start your summary with an introductiion paragraph that overviews the topic.
    4. In bulletpoints lis the main points of this text.
    5. End the summary with a conclusion.
    '''

    print('Starting the summarization process ...', end ='')
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role':'system', 'content': system_prompt},
            {'role':'user','content':prompt}
        ],
        max_tokens=2048,
        temperature=1
    )
    print('Summarization is completed!')
    res=response.choices[0].message.content
    return res

In [35]:
transcribe_file = transcribe('Sample03.m4a')
sentiment_result=analyze_sentiment(transcribe_file)
print(f'The sentiment result is {sentiment_result}')
summary = summarize(transcribe_file)
print('\n')
print(f'The summary of your voice memo is: {summary}')

Starting transcription ...Transcription Completed!
The sentiment result is Neutral
Starting the summarization process ...Summarization is completed!


The summary of your voice memo is: **Summary: Project Planning Meeting Updates**

**Introduction:**
In a project planning meeting, team members give updates on client feedback, design deliverables, and budget projections to solidify the timeline for the next two weeks.

**Key Points:**
- Cindy recaps client feedback: Clients liked design concept but requested minor tweaks.
- Mark updates on design deliverable: Design team is finalized, and deliverables will be ready for review by the end of the day.
- Sarah reports on budget projection: No major changes, everything is on track.
- Revised timeline incorporating updates to be circulated to the team by lunchtime.
- Potential roadblocks discussed: Possible delays due to extensive revisions after the initial review.

**Conclusion:**
The project planning meeting addressed updates on client fee

### Text-to-Speech (TTS) API

In [36]:
text='''
    I'm not sure what to do today. Feeling kind of down and unmotivated. Maybe it's just the weather. But honestly, 
    I think those comments from yesterday are bothering me more than I realized. Should I try to address it, or just let it go? 
    Maybe some exercise would help clear my head and get me focused. It's been a while since I went for a run. 
    Maybe I'll hit the park later this afternoon. Fresh air and some physical activity might be just what I need.  
    I know I shouldn't let things get to me like this. But sometimes it's hard to shake off negative feelings.  
    Well, dwelling on it won't help. Maybe I'll make a list of things I'm grateful for. 
    That usually helps shift my perspective. Okay, let's get started on that...
'''

In [37]:
response = client.audio.speech.create(
    model = 'tts-1',
    voice = 'fable',
    input = text
)

response.stream_to_file('tts.mp3')

/var/folders/gq/6gkq7lc97m7gd82nff7t_xn80000gp/T/ipykernel_50092/1591528767.py:7: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file('tts.mp3')


In [38]:
import IPython
IPython.display.Audio('tts.mp3')

### LLM Chaining

In [39]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [40]:
transcriber = OpenAI(temperature=0)
summarizer = OpenAI(temperature=1.5)

In [41]:
transcribe_template = PromptTemplate(
    input_variables=["audio_file"], 
    template="Transcribe the audio in this file: {audio_file}"
)
summarize_template = PromptTemplate(
    input_variables=["text"], 
    template="Summarize the following text: {text}"
)

In [42]:
transcribe_chain = LLMChain(llm=transcriber, prompt=transcribe_template)
summarize_chain = LLMChain(llm=summarizer, prompt=summarize_template)

In [47]:
def full_analysis(audio_file):
    transcription = transcribe_chain.run(audio_file=audio_file)
    summary = summarize_chain.run(text=transcription)
    return summary

In [48]:
audio_file = "Sample03.m4a"
result=full_analysis(audio_file)
print("Summary:", result)

Summary: Speaker 1 and Speaker 2 had a typical check-in conversation about their day, with Speaker 1 discussing their meetings and completing a project and Speaker 2 mentioning errands and relaxing later on.

